In [20]:
#final

import pandas as pd
import os
import csv

def draw(i):
    import pandas as pd

    df = pd.read_csv("database_size.csv")
    df.fileSize.plot(figsize =(10,5), color = 'b')
    if i == 1: df.fileSizeH.plot(figsize=(10, 5), color = 'y')

    print("Blau = lz77")
    print("Gelb = huffmann")

def findLowest(b):
    import pandas as pd
    smallest = 100000000000000000000000000000000000000000


    df = pd.read_csv('database_size.csv')
    i = 0

    if b == 1:    
        while i < len(df.index):
            if i == 0: 
                smallest = int(df.at[0, 'fileSizeH'])
                pSize = int(df.at[0, 'previewWindowSize'])
                sSize = int(df.at[0, 'searchWindowSize'])
            else:
                if int(df.at[i, 'fileSizeH']) < smallest:
                    smallest = int(df.at[i, 'fileSizeH'])
                    pSize = int(df.at[i, 'previewWindowSize'])
                    sSize = int(df.at[i, 'searchWindowSize'])
            i += 1
    if b == 0:
        while i < len(df.index):
            if i == 0: 
                smallest = int(df.at[0, 'fileSize'])
                pSize = int(df.at[0, 'previewWindowSize'])
                sSize = int(df.at[0, 'searchWindowSize'])
            else:
                if int(df.at[i, 'fileSize']) < smallest:
                    smallest = int(df.at[i, 'fileSize'])
                    pSize = int(df.at[i, 'previewWindowSize'])
                    sSize = int(df.at[i, 'searchWindowSize'])
            i += 1 

    print('SmallestFile:', smallest)
    print('searchWindowSize: ', sSize)
    print('previewWindowSize: ', pSize)

def huffmann(isAutomatic, fileName):
    # Text Eingabefeld 
    
    
    
    if isAutomatic == False: 
        Input = int(input('[1] use File, [2] enter string'))
        if Input == 1:
            Input == input('enter File Name: ')
            with open(Input, 'r') as f:
                string = f.read()
        elif Input == 2:
            string = list(input("Text :"))
    elif isAutomatic == True:
        with open(fileName, 'r') as f:
            string = f.read()


    # Knotenpunkte erstellen 
    class NodeTree(object):

        def __init__(self, left=None, right=None):
            self.left = left
            self.right = right

        def children(self):
            return (self.left, self.right)

        def nodes(self):
            return (self.left, self.right)

        def __str__(self):
            return 'Lückenfüller?' % (self.left, self.right)


    # Haupfunktion zur implementierung des Huffman Codes 
    def huffman_code_tree(node, left=True, binString=''):
        if type(node) is str:
            return {node: binString}
        (l, r) = node.children()
        d = dict()
        d.update(huffman_code_tree(l, True, binString + '0'))
        d.update(huffman_code_tree(r, False, binString + '1'))
        return d


    # Berechnung der Häufigkeit 
    freq = {}
    for c in string:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1

    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

    nodes = freq

    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))

        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    huffmanCode = huffman_code_tree(nodes[0][0])

    #print(' Char | Huffman code ')
    #print('----------------------')
    #for (char, frequency) in freq:
    #    print(' %-4r |%12s' % (char, huffmanCode[char]))
    output = open("huffmann.txt", "w+")
    for (char, frequency) in freq:
        output.write(' %-4r |%12s' % (char, huffmanCode[char]) + '\n')
    
    arr = list(string)
    joined = []

    for c in arr:
      joined.append(huffmanCode[c])
    #print(''.join(joined))
    output.write(''.join(joined))

    
    output.close()

def lz77(searchWindowSize, previewWindowSize, filename, isAutomatic):
    
    # s1 == Inhalt beide Fenster, s2 == Inhalt Lookahead
    def longest_common_substring(s1, s2):
      maxLongest = 0
      offset = 0
      # len(s1) == Endposition der Fenster; i == Position => range = gesamte Länge => für jede Ziffer in Länge beider Fenster
      for i in range(0, len(s1)):
        longest = 0  # Variablen werden zurückgesetzt
        if i == len(s1) - len(s2) - 2:  # Fehler
          break
        for j in range(0,
                       len(s2)):  # für jede Zahl in der Länge des Preview Windows
          if (
              i + j < len(s1)
          ):  # wenn position gesamt + position preview kleinber als die Gesamtlänge => solange man sich innerhalb des bekannten Bereichs befindet
            if s1[i + j] == s2[j]:  # wenn  beide Positionen übereinstimmen
              longest = longest + 1  # variablen festlegen
              if (maxLongest <
                  longest):  # wenbn kein längerer String gefunden wurde
                maxLongest = longest
                offset = i  # offset entspricht der Position im gesamten String
            else:  # Fehler
              break
          else:  # Fehler
            break
      return maxLongest, offset  # return == Rückgabewerte der Funktion


    def encode_lz77(text, searchWindowSize, previewWindowSize):
      encodedNumbers = []  #{
      encodedSizes = []  #  Variablen zurücksetzten//definieren
      encodedLetters = []  #}
      i = 0  # i = Position im Text i == 0, weil 0 Position 1 ist
      while i < len(text):  # solange man sich im bekannten Bereich befinfet
        if i < previewWindowSize:  # solange es wegen der Position der Fenster noch keine Übereinstimmungen gibt
          encodedNumbers.append(0)  #{
          encodedSizes.append(
            0)  #  append = zu Array hinzufügen, hier wird die "Tabelle" definiert
          encodedLetters.append(text[i])  #}
          i += 1  # Position wird "aktualisiert"
        else:
          previewString = text[
            i:i + previewWindowSize]  # dem Preview Window Inhalt zuweisen
          searchWindowOffset = 0  # Position des searchWindow wird zu Null gesetzt
          if (i < searchWindowSize):
            searchWindowOffset = i
          else:
            searchWindowOffset = searchWindowSize
          searchString = text[i - searchWindowOffset:i]
          result = longest_common_substring(
            searchString + previewString, previewString
          )  # Funktion aufrufen, result = Rückgabewert longest_common_substring. longest_common_substring gibt den längsten übereinstimmenden Text zurück
          nextLetter = ''  # Variable "leeren"/"zurücksetzen"
          if (result[0] == len(previewString)):  # {
            if (i + result[0] == len(text)):
              nextLetter = ''
            else:
              nextLetter = text[i + previewWindowSize]
          else:
            nextLetter = previewString[result[0]]
          if (result[0] == 0):
            encodedNumbers.append(0)
            encodedSizes.append(0)
            encodedLetters.append(nextLetter)
          else:
            encodedNumbers.append(searchWindowOffset - result[1])
            encodedSizes.append(result[0])
            encodedLetters.append(nextLetter)
          i = i + result[0] + 1  # } Formattierung
      return encodedNumbers, encodedSizes, encodedLetters


    def decode_lz77(encodedNumbers, encodedSizes, encodedLetters):
      i = 0  # i entspricht Position im encodeten String
      decodedString = []  # "Ergebnis", encodeter Text
      while i < len(encodedNumbers):  # solange man sich im Code befindet
        if (encodedNumbers[i] == 0):
          decodedString.append(encodedLetters[i])
        else:
          currentSize = len(decodedString)
          for j in range(0, encodedSizes[i]):
            decodedString.append(decodedString[currentSize - encodedNumbers[i] +
                                               j])
          decodedString.append(encodedLetters[i])
        i += 1
      return decodedString  # Ergebnis

    
    #========================================================================================================
    #Start "Main-Loop"
    if isAutomatic == False:
        print("LZ77 Compression Algorithm")
        print("=================================================================")
        h = int(
          input(
            "Enter 1 if you want to enter input in command window, 2 if you are using some file:"
          ))
        if h == 1:
          stringToEncode = input("Enter the string you want to compress:")
        elif h == 2:
          file = input("Enter the filename:")
          with open(file, 'r') as f:
            stringToEncode = f.read()
        else:
          print("You entered invalid input")
        print("Enetered string is:", stringToEncode)
        searchWindowSize = int(input("Enter the Search Window Size:"))
        previewWindowSize = int(input("Enter the Preview Window Size:"))
    else:
        file = filename
        with open(file, 'r') as f:
            stringToEncode = f.read()
    
    
    [encodedNumbers, encodedSizes, encodedLetters] = encode_lz77(stringToEncode, searchWindowSize,previewWindowSize)
    a = [encodedNumbers, encodedSizes, encodedLetters]
    #print("Compressed file generated as compressed.txt")
    output = open("compressed.txt", "w+")
    output.write(str(a))
    if isAutomatic == False: print("Encoded string: ", end="")
    i = 0
    while i < len(encodedNumbers) and isAutomatic == False:  # Ausgabe der "Tabelle"
      print("{",
            encodedNumbers[i],
            ":",
            encodedSizes[i],
            ":",
            encodedLetters[i],
            "}",
            end=" ")
      i += 1
    if isAutomatic == False: print("\n")
    decodedString = decode_lz77(encodedNumbers, encodedSizes, encodedLetters)
    if isAutomatic == False: print("Decoded string:", "".join(decodedString))

def create_db():
    
    filePath = r'compressed.txt'
    sampleFilePath = r'sampleFile.txt'

    Input = int(input("[1] lz77 testen, [2] lz77 -> huffmann testen, [3] huffman -> lz77 testen"))
    
    f = open('database_size.csv', 'w+')
    writer = csv.writer(f)
    
    #mainloop
    range_preview = int(input("In welcher Reichweite sollen die Wiederholungen der Preview_Window_Size sich bewegen? "))
    range_search = int(input("In welcher Reichweite sollen die Wiederholungen der Search_Window size sich bewegen? "))
    
    if Input == 1: writer.writerow(['previewWindowSize']+['searchWindowSize']+['fileSize'])
    if Input == 2 or Input == 3: writer.writerow(['previewWindowSize']+['searchWindowSize']+['fileSize']+['fileSizeH'])

    searchWindowSize = 1
    previewWindowSize = 1

    s = 1
    p = 1
    while s <= range_search:
        while p <= range_preview:
            if Input == 1:
              lz77(s, p, sampleFilePath, True)
              fileSize = os.path.getsize(filePath)
              writer.writerow([p]+[s]+[fileSize])
            
            if Input == 2:
              lz77(s, p, sampleFilePath, True)
              huffmann(True, 'compressed.txt')
              
              fileSizeH = os.path.getsize(r'huffmann.txt')
              fileSize = os.path.getsize(r'compressed.txt')

              writer.writerow([p]+[s]+[fileSize]+[fileSizeH])

            if Input == 3:
              huffmann(True, 'sampleFile.txt')
              lz77(s, p, 'huffmann.txt', True)

              fileSizeH = os.path.getsize(r'huffmann.txt')
              fileSize = os.path.getsize(r'compressed.txt')

              writer.writerow([p]+[s]+[fileSize]+[fileSizeH])
              
            p += 1
            
        p = 1
        s += 1
    
    f.close()
    
    
    if Input == 1:
        findLowest(0)
        draw(0)
    elif Input == 2:
        findLowest(1)
        draw(1)
    elif Input == 3:
        findLowest(0)
        draw(1)

#Dialog
Input = int(input("[1] Datenbank, [2] Komprimierung testen"))
if Input == 1:
    create_db()
    #os.remove('database_size.csv')
elif Input == 2:
    Input = int(input("[1] lz77 testen, [2] huffmann testen, [3] lz77 -> huffmann, [4] huffman -> lz77"))
    if Input == 1:
        lz77(1, 1, "None", False)
    elif Input == 2:
        huffmann(False, "None")
    elif Input == 3:
        p = int(input("Größe lz77 Preview: "))
        s = int(input("Größe lz77 Search: "))
        filename = input("zu komprimierende Datei: ")
        lz77(s, p, filename, True)
        huffmann(True, "compressed.txt")
    elif Input == 4:
      p = int(input("Größe lz77 Preview: "))
      s = int(input("Größe lz77 Search: "))
      filename = input("zu komprimierende Datei: ")
      huffmann(True, 'sampleFile.txt')
      lz77(s, p, filename, True)


ValueError: invalid literal for int() with base 10: ''